In [2]:
pip install lxml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
from scipy import signal
import numpy as np
import matplotlib as plt
import time

import matplotlib.pyplot as plt
import warnings
import re

# Ignore all warnings
warnings.filterwarnings("ignore")

In [49]:
Combine = pd.read_csv("data/combine.csv")
Team_Stats = pd.read_csv("data/team_stats_2003_2023.csv")

In [50]:
Team_Stats = Team_Stats[Team_Stats["year"] != 2023]

## Webscrape all of the years for from pro-football reference that we are using

In [51]:
website_begining = 'https://www.pro-football-reference.com/years/'
years = Team_Stats["year"].unique()
years = years
positions = ['passing', 'rushing', 'receiving']
years

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [8]:
for pos in positions:
    print(pos)
    for year in years:
        link = website_begining + str(year) +'/'+ pos + '.htm'
        
        df = pd.read_html(link)
        
        csv_name = 'data/'+str(year) + '_' + pos + '.csv'
        df[0].to_csv(csv_name, sep=',', index=False, encoding='utf-8')
        time.sleep(4.2) #wait four seconds
        print(year)

passing
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
rushing
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
receiving
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


## Concat the tables together to put them all into one data frame and take out unnessary data from player name

In [52]:
passing = pd.read_csv('data/2003_passing.csv')

passing["Year"] = years[0]
for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_passing.csv')
    temp["Year"] = year
    passing = pd.concat([passing, temp])
    
passing = passing[passing["Pos"] == "QB"]
passing.reset_index(drop=True, inplace=True)

In [53]:
index = 0
for name in passing["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    passing["Player"][index] = front + back
    
    index += 1

In [54]:
rushing = pd.read_csv('data/2003_rushing.csv')

rushing["Year"] = years[0]

for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_rushing.csv')
    temp["Year"] = year
    rushing = pd.concat([rushing, temp])
    
    
rushing.columns = rushing.iloc[0]
rushing = rushing.drop(0)
rushing.rename(columns={2003: 'Year'}, inplace=True)

rushing = rushing[rushing["Pos"] == "RB"]
rushing.reset_index(drop=True, inplace=True)

In [55]:
index = 0
for name in rushing["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    rushing["Player"][index] = front + back
    
    index += 1


In [56]:
receiving = pd.read_csv('data/2003_receiving.csv')

receiving["Year"] = years[0]
for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_receiving.csv')
    temp["Year"] = year
    receiving = pd.concat([receiving, temp])
    
receiving = receiving[receiving["Pos"] == "WR"]
receiving.reset_index(drop=True, inplace=True)

In [57]:
index = 0
for name in receiving["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    receiving["Player"][index] = front + back
    
    index += 1
    

## Putting Combine Data into Positional Dfs

In [58]:
for col in Combine.columns:
    passing[col] = 0
    rushing[col] = 0
    receiving[col] = 0

In [59]:
passing_names = passing["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in passing_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        passing.loc[passing["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(passing_names))) + "% of the players had Combine data")
passing = passing[passing["hsCountry"] != 0]

70% of the players had Combine data


In [60]:
rushing_names = rushing["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in rushing_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        rushing.loc[rushing["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(rushing_names))) + "% of the players had Combine data")
rushing = rushing[rushing["hsCountry"] != 0]

57% of the players had Combine data


In [61]:
receiving_names = receiving["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in receiving_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        receiving.loc[receiving["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(receiving_names))) + "% of the players had Combine data")
receiving = receiving[receiving["hsCountry"] != 0]

49% of the players had Combine data


In [62]:
passing.drop(columns=['Rk'], inplace=True)
rushing.drop(columns=['Rk'], inplace=True)
receiving.drop(columns=['Rk'], inplace=True)

## Putting the team data into Positional Dfs

In [63]:
index = 0
for team in Team_Stats["team"]:
    team_split = team.split(" ")
    simple = team_split[-1]
    
    Team_Stats["team"][index] = simple
    index += 1

In [64]:
Team_Stats["team"] = Team_Stats["team"].replace("Redskins", "Commanders")
Team_Stats["team"] = Team_Stats["team"].replace("Team", "Commanders")

In [65]:
df_list = [passing, rushing, receiving]

In [66]:
for df in df_list:
    df["Tm"] = df["Tm"].replace("MIN", "Vikings")
    df["Tm"] = df["Tm"].replace("KAN", "Chiefs")
    df["Tm"] = df["Tm"].replace("IND", "Colts")
    df["Tm"] = df["Tm"].replace("DEN", "Broncos")
    df["Tm"] = df["Tm"].replace("GNB", "Packers")
    df["Tm"] = df["Tm"].replace("STL", "Rams")#
    df["Tm"] = df["Tm"].replace("PHI", "Eagles")
    df["Tm"] = df["Tm"].replace("NOR", "Steelers")#
    df["Tm"] = df["Tm"].replace("NWE", "Patriots")
    df["Tm"] = df["Tm"].replace("DAL", "Cowboys")

    df["Tm"] = df["Tm"].replace("HOU", "Texans")
    df["Tm"] = df["Tm"].replace("OAK", "Raiders")
    df["Tm"] = df["Tm"].replace("SDG", "Chargers")
    df["Tm"] = df["Tm"].replace("DET", "Lions")
    df["Tm"] = df["Tm"].replace("JAX", "Jaguars")
    df["Tm"] = df["Tm"].replace("BUF", "Bills")
    df["Tm"] = df["Tm"].replace("CIN", "Bengals")
    df["Tm"] = df["Tm"].replace("NYJ", "Jets")
    df["Tm"] = df["Tm"].replace("PIT", "Steelers")
    df["Tm"] = df["Tm"].replace("BAL", "Ravens")

    df["Tm"] = df["Tm"].replace("ARI", "Cardinals")
    df["Tm"] = df["Tm"].replace("ATL", "Falcons")
    df["Tm"] = df["Tm"].replace("IND", "Colts")
    df["Tm"] = df["Tm"].replace("SFO", "49ers")
    df["Tm"] = df["Tm"].replace("CLE", "Browns")
    df["Tm"] = df["Tm"].replace("TEN", "Titans")
    df["Tm"] = df["Tm"].replace("WAS", "Commanders")
    df["Tm"] = df["Tm"].replace("NYG", "Giants")
    df["Tm"] = df["Tm"].replace("CHI", "Bears")
    df["Tm"] = df["Tm"].replace("TAM", "Buccaneers")

    df["Tm"] = df["Tm"].replace("MIA", "Dolphins")
    df["Tm"] = df["Tm"].replace("CAR", "Panthers")
    df["Tm"] = df["Tm"].replace("SEA", "Seahawks")
    df["Tm"] = df["Tm"].replace("2TM", "Take_out")
    df["Tm"] = df["Tm"].replace("LAR", "Rams")
    df["Tm"] = df["Tm"].replace("LAC", "Chargers")
    df["Tm"] = df["Tm"].replace("LVR", "Raiders")
    
    df["Tm"] = df["Tm"].replace("Team", "Commanders")

In [67]:
passing = passing[(passing["Tm"] != "Take_out") & (passing["Tm"] != "3TM")]
rushing = rushing[(rushing["Tm"] != "Take_out") & (rushing["Tm"] != "3TM" )]
receiving = receiving[(receiving["Tm"] != "Take_out") & (receiving["Tm"] != "3TM")]

In [68]:
pos_team_cols =Team_Stats.columns[2:]
for col in pos_team_cols:
    passing["team_" + col] = 0
    rushing["team_" + col] = 0
    receiving["team_" + col] = 0
    

passing.reset_index(drop=True, inplace=True)
rushing.reset_index(drop=True, inplace=True)
receiving.reset_index(drop=True, inplace=True)

In [69]:
df_list = [passing, rushing, receiving]
team_years = Team_Stats["year"].unique()
for df in df_list:
    for year in team_years:
        for team in Team_Stats["team"].unique():
            team_values = Team_Stats[(Team_Stats["team"]==team) & (Team_Stats["year"]==year)].values.tolist()[0]
            df.loc[(df["Tm"] == team) & (df["Year"] == year), "team_" + Team_Stats.columns[2:]] = team_values[2:]

## Taking out unnessary Columns

In [70]:
df_list = [passing, rushing, receiving]
questionable_cols = []
for df in df_list:
    question = 0
    question_temp = []
    for col in df.columns:
        if type(df[col][0]) != float or type(df[col][0]) != int:
            if col != 'Player' and col != 'Tm':
                try:
                    df[col] = df[col].astype(float)
                except:
                    question_temp.append(col)
                    question += 1

    print(100*question/len(df.columns), "% was questionable")
    print(question_temp)
    questionable_cols.append(question_temp)

18.0 % was questionable
['Pos', 'QBrec', 'combinePosition', 'nameFirst', 'nameLast', 'nameFull', 'position', 'nflId', 'college', 'dob', 'playerProfileUrl', 'homeCity', 'homeState', 'homeCountry', 'highSchool', 'hsCity', 'hsState', 'hsCountry']
20.238095238095237 % was questionable
['Pos', 'combinePosition', 'nameFirst', 'nameLast', 'nameFull', 'position', 'nflId', 'college', 'dob', 'playerProfileUrl', 'homeCity', 'homeState', 'homeCountry', 'highSchool', 'hsCity', 'hsState', 'hsCountry']
20.454545454545453 % was questionable
['Pos', 'Ctch%', 'combinePosition', 'nameFirst', 'nameLast', 'nameFull', 'position', 'nflId', 'college', 'dob', 'playerProfileUrl', 'homeCity', 'homeState', 'homeCountry', 'highSchool', 'hsCity', 'hsState', 'hsCountry']


In [71]:
passing = passing.drop(columns = [questionable_cols[0]][0])
rushing = rushing.drop(columns = [questionable_cols[1]][0])
receiving = receiving.drop(columns = [questionable_cols[2]][0])

## Deal with Nan values

In [72]:
df_list = [passing, rushing, receiving]
nan_columns = []
for df in df_list:
    temp_nan = []
    total = 0
    for col in df.columns:
        column_nan_percentage = df[col].isna().mean()
        if column_nan_percentage >= 0.05:
            print(col, column_nan_percentage*100)
            total += 1
            temp_nan.append(col)
    nan_columns.append(temp_nan)
    print(total/len(df.columns))

4QC 45.950554134697356
GWD 45.950554134697356
QBR 15.515771526001704
combineHand 88.57630008525149
combineArm 88.7468030690537
combine40yd 6.138107416879795
combineVert 23.017902813299234
combineBench 94.8849104859335
combineShuttle 27.621483375959077
combineBroad 24.21142369991475
combine3cone 31.54305200341006
combine60ydShuttle 98.80647911338448
combineWonderlic 30.434782608695656
team_mov 48.4228473998295
team_ties 51.57715260017051
0.18292682926829268
combineHand 92.90780141843972
combineArm 93.14420803782507
combine40yd 6.796690307328605
combineVert 11.93853427895981
combineBench 18.55791962174941
combineShuttle 33.92434988179669
combineBroad 15.425531914893616
combine3cone 39.00709219858156
combine60ydShuttle 97.04491725768321
combineWonderlic 93.91252955082743
team_mov 50.945626477541374
team_ties 49.054373522458626
0.1791044776119403
combineHand 88.15331010452961
combineArm 88.93728222996515
combine40yd 6.837979094076655
combineVert 13.937282229965156
combineBench 48.867595818

In [73]:
passing = passing.drop(columns = [nan_columns[0]][0])
rushing = rushing.drop(columns = [nan_columns[1]][0])
receiving = receiving.drop(columns = [nan_columns[2]][0])

## Replace rest of Nan with column average

In [74]:
df_list = [passing, rushing, receiving]
for df in df_list:
    total = 0
    for col in df.columns:
        column_nan_percentage = df[col].isna().mean()
        if column_nan_percentage > 0:
            
            column_average = df[col].mean()


            df[col] = df[col].fillna(column_average)
            print(col)

Y/C
ageAtDraft
ageAtDraft
Tgt
Y/R
Succ%
Y/Tgt
Fmb
ageAtDraft


# WebScrape Fantasy Football Rankings from Pro Football Reference

In [75]:
website_begining = 'https://www.pro-football-reference.com/years/'
years = Team_Stats["year"].unique()
years_fantasy = years + 1
positions = ['passing', 'rushing', 'receiving']
years_fantasy

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [196]:
for year in years_fantasy:
    link = website_begining + str(year) + '/fantasy.htm'

    df = pd.read_html(link)

    csv_name = 'data/'+str(year) + '_Fantasy_Score.csv'
    df[0].to_csv(csv_name, sep=',', index=False, encoding='utf-8')
    time.sleep(4.2) #wait four seconds
    print(year)

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [76]:
Fantasy = pd.read_csv('data/2003_Fantasy_Score.csv')

Fantasy["Year"] = years_fantasy[0]
for year in years_fantasy[1:]:
    temp = pd.read_csv('data/' + str(year) + '_Fantasy_Score.csv')
    temp["Year"] = year
    Fantasy = pd.concat([Fantasy, temp])
    
    
    
Fantasy.columns = Fantasy.iloc[0]
Fantasy = Fantasy.drop(0)
Fantasy.rename(columns={years_fantasy[0]: 'Year'}, inplace=True)


Fantasy = Fantasy[(Fantasy["FantPos"] == "QB") | (Fantasy["FantPos"] == "WR") | (Fantasy["FantPos"] == "RB")]

Fantasy.reset_index(drop=True, inplace=True)

In [77]:
index = 0
for name in Fantasy["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    Fantasy["Player"][index] = front + back
    
    index += 1
    
keep_columns = ['Player', 'PosRank', 'Year']

Fantasy = Fantasy[keep_columns]

# Add Rankings to Dfs

In [78]:

passing["Next_Year_PosRank"] = 0
rushing["Next_Year_PosRank"] = 0
receiving["Next_Year_PosRank"] = 0
    

passing.reset_index(drop=True, inplace=True)
rushing.reset_index(drop=True, inplace=True)
receiving.reset_index(drop=True, inplace=True)

In [79]:

fantasy_names = Fantasy["Player"].unique()
total = 0
has_data = 0
for df in df_list:
    df_names = df["Player"].unique()
    total += len(df_names)
    for name in df_names:
        if name in fantasy_names:
            for year in years:
                try:
                    fantasy_value = int(Fantasy[(Fantasy["Player"] == name) & (Fantasy["Year"] == year + 1)]["PosRank"])
                
                    df.loc[(df["Player"] == name) & (df["Year"] == year), ['Next_Year_PosRank']] = fantasy_value
        
                #df[(df["Player"] == name) & (df["Year"] == year)]['PosRank'] = fantasy_value

                    has_data += 1
                    total += 1
                except:
                    pass
    
    
print(str(int(100*has_data/total)) + "% of the players had PosRank data")

82% of the players had PosRank data


# Normalizing all of the columns using Z-Score normalization

In [81]:
from sklearn.preprocessing import StandardScaler
scaler_standard = StandardScaler()

df_list = [passing, rushing, receiving]
for df in df_list:
    for col in df.columns:
        if (df[col].dtype == float or df[col].dtype == int) and col != "Next_Year_PosRank":
            df[col] = scaler_standard.fit_transform(df[[col]])


In [82]:
passing

,Player,Tm,Age,G,GS,Cmp,Att,Cmp%,Yds,TD,...,team_rush_td,team_rush_yds_per_att,team_rush_fd,team_penalties,team_penalties_yds,team_pen_fd,team_score_pct,team_turnover_pct,team_exp_pts_tot,Next_Year_PosRank
0,Peyton Manning,Colts,-0.344709,1.229417,1.297227,1.569013,1.419425,0.614432,1.506824,1.547104,...,0.606368,-1.121308,0.287212,-0.636309,-1.345998,0.581065,1.940346,-0.796676,0.322892,2
1,Trent Green,Chiefs,0.981007,1.229417,1.297227,1.219180,1.222175,0.305001,1.366436,1.112370,...,3.855103,0.266377,1.081787,-1.443082,-1.096188,-0.095249,0.751906,-1.320067,0.745546,3
2,Marc Bulger,Rams,-0.565661,1.050495,1.142698,1.262016,1.263460,0.312935,1.246982,0.938477,...,1.215506,-1.352589,-0.060414,-0.636309,-1.311302,-0.095249,1.167860,1.235316,-0.885263,10
3,Brad Johnson,Buccaneers,1.422912,1.229417,1.297227,1.390527,1.437774,0.225660,1.226047,1.286264,...,-1.627137,-0.658746,-0.606685,0.915180,1.721115,0.445802,-1.149598,0.989014,-1.350900,12
4,Tom Brady,Patriots,-0.565661,1.229417,1.297227,1.126367,1.240524,0.074911,1.108441,1.025423,...,-0.814953,-1.815151,-0.358380,0.542822,0.985563,-0.230512,-0.793066,-0.458010,-0.697926,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,Blaine Gabbert,Buccaneers,0.981007,-1.454403,-1.175237,-1.094005,-1.140242,1.249163,-1.102679,-0.887406,...,-1.627137,-1.815151,-0.954311,-0.201892,-0.436966,-0.095249,-0.183990,-0.765888,0.447756,66
1169,Brandon Allen,Bengals,0.318149,-1.454403,-1.175237,-1.115423,-1.163178,3.232697,-1.106989,-0.974352,...,0.200276,-0.890027,-0.110075,-1.318963,-1.616625,1.663167,1.227282,-0.950614,1.668822,0
1170,Josh Johnson,49ers,1.643865,-1.275482,-1.175237,-1.129702,-1.167766,-0.734371,-1.114378,-0.974352,...,1.418552,1.191501,1.677718,-0.512189,-0.874134,-0.095249,1.212427,-1.135341,1.782690,0
1171,Case Keenum,Bills,1.201960,-1.275482,-1.175237,-1.122563,-1.144830,-2.432276,-1.115609,-0.974352,...,0.403322,2.347905,1.479074,-0.636309,-1.019857,-0.095249,1.747225,0.742712,1.750787,64


In [83]:
print("Final Shapes of DataFrames:\n")
print('Passing Observations:', passing.shape[0], '       Passing Parameters:', passing.shape[1] - 2, '\n')
print('Rushing Observations:', rushing.shape[0], '       Rushing Parameters:', rushing.shape[1] - 2, '\n')
print('Receiving Observations:', receiving.shape[0], '     Receiving Parameters:', receiving.shape[1] - 2, '\n')

Final Shapes of DataFrames:

Passing Observations: 1173        Passing Parameters: 66 

Rushing Observations: 1692        Rushing Parameters: 54 

Receiving Observations: 2296      Receiving Parameters: 57 



In [84]:
passing.to_csv('data/Cleaned_Passing.csv', sep=',', index=False, encoding='utf-8')
rushing.to_csv('data/Cleaned_Rushing.csv', sep=',', index=False, encoding='utf-8')
receiving.to_csv('data/Cleaned_Receiving.csv', sep=',', index=False, encoding='utf-8')